In [1]:
import os

import numpy as np
import pandas as pd
import xlrd

pd.set_option("display.max_rows", 50000)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
stocks = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Stocks_DS_tab_delimited.dat", sep="\t", na_values=' ', parse_dates=["datadate"])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,6,11,14,15,16,17,18,19,20,21,38,42,43,44,45,50,52,53,60,64,65,70,71,74,75) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
fundamentals = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Fundamentals_Full.csv", na_values=' ')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,25,29,916,917,919,920,926,935,937,940,941,942,943,945,955,956,959,962,965,970,978,981,984,989,992,993,997,998,1008,1009,1014,1015,1016,1017,1018,1019,1020,1021,1024,1025,1028,1029,1031,1033,1034,1035,1036,1040,1041,1042,1052,1053,1054,1055,1056,1057,1058,1059,1066,1071,1072,1074,1075,1076,1077,1081,1084,1091,1094,1096,1098,1101,1102,1103,1105,1106,1107,1108,1116,1117,1119,1120,1121,1122,1123,1124,1125,1735,1736,1737,1742,1756,1757,1762) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
ratings = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Ratings_Full.csv", na_values=' ')

In [6]:
securities = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Securities_Full.csv", na_values=' ')

In [7]:
settlements = pd.read_excel("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Settlements.xlsx", na_values="#NULL!")

In [8]:
def excel_to_dictionary(sheetname, filename="/content/drive/MyDrive/Fiverr/PROJECT_MIMI/DataDictionary.xlsx"):
    abbreviation_dict = dict()
    
    abbreviation = xlrd.open_workbook(os.path.abspath(filename))
    abbreviation_sheet = abbreviation.sheet_by_name(sheetname)
    
    for i in range(1, abbreviation_sheet.nrows):
        abb = abbreviation_sheet.cell(i, 0).value
        abb_definition = abbreviation_sheet.cell(i, 1).value
        abbreviation_dict[abb] = abb_definition
        
    return abbreviation_dict

In [9]:
stocks_dict = excel_to_dictionary(sheetname="Stocks")
fundamentals_dict = excel_to_dictionary(sheetname="Fundamentals")
ratings_dict = excel_to_dictionary(sheetname="Ratings")
securities_dict = excel_to_dictionary(sheetname="Securities")

In [10]:
for key, value in stocks_dict.items():
  if key in ["prccd", "prchd", "prcld", "prcod"]:
    print(value)

  # if value == "Data Date - Dividends":
  #   print(key)

Price - Close - Daily
Price - High - Daily
Price - Low - Daily
Price - Open - Daily


In [11]:
def rename_columns(df, df_dict):
  df.rename(columns=df_dict, inplace=True)
  df.columns = [column.lower().replace(" ", "_") for column in df.columns]
  return df

In [12]:
stocks = rename_columns(stocks, stocks_dict)
fundamentals = rename_columns(fundamentals, fundamentals_dict)
ratings = rename_columns(ratings, ratings_dict)
securities = rename_columns(securities, securities_dict)

In [13]:
for name, df in zip(["stocks", "fundamentals", "ratings", "securities", "settlements"], [stocks, fundamentals, ratings, securities, settlements]):
  print("{}: {}\n".format(name, df.shape[0]))

stocks: 4187047

fundamentals: 61429

ratings: 502561

securities: 915386

settlements: 1892



In [14]:
initial_stocks = stocks["global_company_key"].nunique()
print("The stocks data file has {} companies while the fundamentals file has {} companies".format(initial_stocks, fundamentals["global_company_key"].nunique()))

The stocks data file has 4632 companies while the fundamentals file has 10555 companies


In [15]:
stocks["year"] = stocks["data_date_-_dividends"].apply(lambda x: x.strftime("%Y"))

In [16]:
# stocks.isnull().sum()

In [17]:
shared_stocks = list()

for column in sorted(stocks.columns):
  if column in sorted(fundamentals.columns):
    shared_stocks.append(column)

# sorted(shared_stocks)

In [18]:
shared_stock_companies = list(set(fundamentals["global_company_key"].unique()) & set(stocks["global_company_key"].unique()))

In [19]:
stocks = stocks[stocks["global_company_key"].isin(shared_stock_companies)]

In [20]:
print("Stocks has dropped from {} to {} companies".format(initial_stocks, stocks["global_company_key"].nunique()))

Stocks has dropped from 4632 to 2462 companies


In [21]:
# stocks.groupby(['global_company_key',
#  'ticker_symbol',
#  'company_name',
#  'stock_exchange_code',
#  'current_iso_country_code_-_incorporation',
#  'postal_code',
#  's&p_business_description',
#  'city',
#  'company_legal_name',
#  'active/inactive_status_marker',
# #  'county_code',
#  'research_co_reason_for_deletion',
#  'employer_identification_number',
#  'fax_number',
#  'current_fiscal_year_end_month',
#  'gic_groups',
#  'gic_industries',
#  'gic_sectors',
#  'gic_sub-industries',
#  'international,_domestic,_both_indicator',
#  'current_state/province_of_incorporation_code',
#  'current_iso_country_code_-_headquarters',
#  'north_american_industry_classification_code',
#  'phone_number',
# #  'current_primary_issue_tag_-_canada',
# #  'primary_issue_tag_-_rest_of_world',
# #  'current_primary_issue_tag_-_us',
# #  'standard_industry_classification_code',
# #  's&p_industry_sector_code',
# #  's&p_economic_sector_code',
# #  's&p_quality_ranking_-_current',
# #  'state/province',
# #  'stock_ownership_code',
# #  'web_url',
# #  'research_company_deletion_date',
# #  'company_initial_public_offering_date'
 
#  ])["price_-_close_-_daily"].agg("count")